<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/CAFE_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:

from huggingface_hub import notebook_login

notebook_login()

In [3]:

from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:

import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [7]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/CaFE/CaFE_splits/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/CaFE/CaFE_splits/test.json",'r') as f:
    test_data = json.load(f)

In [8]:
train_data

{'09-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-4.wav',
  'length': 3.5731875,
  'emo': 'happiness'},
 '09-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-5.wav',
  'length': 3.8738125,
  'emo': 'happiness'},
 '04-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-6.wav',
  'length': 5.84675,
  'emo': 'happiness'},
 '02-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-1.wav',
  'length': 4.8348125,
  'emo': 'happiness'},
 '10-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-6.wav',
  'length': 4.38675,
  'emo': 'happiness'},
 '10-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-4.wav',
  'length': 4.6013125,
  'emo': 'happiness'},
 '02-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-3.wav',
  'length': 4.6151875,
  'emo': 'happiness'},
 '04-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-4.wav',
  'length': 4.626,
  'emo': 'happiness'},
 '09-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-6.wav',
  'length': 3.7488125,
  'emo': 'happiness'},
 '04-J-1-5': {'wav': 'CaFE_

In [10]:
test_data

{'11-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-6.wav',
  'length': 5.1439375,
  'emo': 'happiness'},
 '11-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-4.wav',
  'length': 4.9079375,
  'emo': 'happiness'},
 '11-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-5.wav',
  'length': 4.6163125,
  'emo': 'happiness'},
 '11-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-1.wav',
  'length': 4.2954375,
  'emo': 'happiness'},
 '11-J-1-2': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-2.wav',
  'length': 4.0303125,
  'emo': 'happiness'},
 '11-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-3.wav',
  'length': 4.2556875,
  'emo': 'happiness'},
 '12-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-6.wav',
  'length': 5.267,
  'emo': 'happiness'},
 '12-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-5.wav',
  'length': 5.34375,
  'emo': 'happiness'},
 '12-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-4.wav',
  'length': 5.4876875,
  'emo': 'happiness'},
 '12-J-1-1': {'wav': 'CaF

In [11]:
%%capture
!git clone https://github.com/aalto-speech/mtkd4ser.git

In [12]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)


In [13]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [14]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
})

In [15]:
def add_base_path(example):
    example['wav'] = '/content/CaFE/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

{'id': '09-J-1-4',
 'wav': '/content/CaFE/CaFE_48k/Happiness/Faible/09-J-1-4.wav',
 'length': 3.5731875,
 'emo': 'happiness'}

In [16]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [17]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
})

In [18]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'])

In [19]:
!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
!pip install tqdm
from tqdm import tqdm


In [20]:
# Metrics functions
def calculate_unweighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='macro')

def calculate_weighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='weighted')

def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# Collate function for DataLoader
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    labels = [int(example["label"]) for example in batch]
    inputs = pad_sequence(inputs, batch_first=True)
    return {"input_values": inputs, "label": torch.tensor(labels)}

# Example helper function to update paths
def update_cafe_path(path):
    return "/content/CaFE/" + path  # Adjust as necessary for your file path

In [21]:
def is_common(label):
    return label in ['anger', 'happiness', 'neutral', 'sadness']

def to_label(label, label2id):
    return label2id[label]


In [22]:
def cafe(session=None):
    all_files = glob.glob("/content/CaFE/CaFE_splits/*")
    cafe_df = pd.DataFrame()

    for file in all_files:
        with open(file, "r") as f:
            data = json.load(f)
            df = pd.DataFrame.from_dict(data, orient="index").reset_index()
            df["set"] = [file.split("/")[-1].split(".")[0] for _ in range(len(df))]
            df = df.rename(columns={'wav': 'audio', 'emo': 'label'})
            df["label_flag"] = df["label"].apply(is_common)
            cafe_df = pd.concat([cafe_df, df], axis=0)

    cafe_df = cafe_df.loc[cafe_df["label_flag"] == 1].reset_index(drop=True)
    cafe_df_train = cafe_df.loc[cafe_df["set"] != "test"][["audio", "label"]]
    cafe_df_test = cafe_df.loc[cafe_df["set"] == "test"][["audio", "label"]]

    cafe_df_train["audio"] = cafe_df_train["audio"].apply(update_cafe_path).values
    cafe_df_test["audio"] = cafe_df_test["audio"].apply(update_cafe_path).values

    labels = ['anger', 'happiness', 'neutral', 'sadness']
    label2id, id2label = {label: str(i) for i, label in enumerate(labels)}, {str(i): label for i, label in enumerate(labels)}

    train_df, test_df = cafe_df_train, cafe_df_test
    train_df["label"] = train_df["label"].apply(to_label, args=(label2id,)).values
    test_df["label"] = test_df["label"].apply(to_label, args=(label2id,)).values

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']]).cast_column("audio", Audio(sampling_rate=16_000))
    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']]).cast_column("audio", Audio(sampling_rate=16_000))

    return label2id, id2label, DatasetDict({'train': train_audio_data, 'test': test_audio_data, 'dev': test_audio_data})

In [23]:
def preprocess_function(examples, feature_extractor):
    max_duration = 20.0
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=int(feature_extractor.sampling_rate * max_duration), padding=True, truncation=True)
    return inputs


In [24]:
def model_ft(label2id, id2label, num_classes, device="cpu"):
    model_name = "facebook/wav2vec2-base"
    model = AutoModelForAudioClassification.from_pretrained(model_name, num_labels=num_classes, label2id=label2id, id2label=id2label)
    model.to(device)
    return model

In [25]:
def pipeline_ft(model, train_loader, valid_loader, test_loader, hyperparam, device):
    SESSION = hyperparam['SESSION']
    LEARNING_RATE = hyperparam['LEARNING_RATE']
    TRAINING = hyperparam['TRAINING']
    N_EPOCHS = hyperparam['N_EPOCHS']
    LINGUALITY = hyperparam['LINGUALITY']
    LANGUAGE = hyperparam['LANGUAGE']

    file_path = f"/content/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}.pth"
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Load checkpoint if exists
    epoch = 1
    if os.path.exists(file_path):
        checkpoint = torch.load(file_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        total_train_loss = checkpoint.get('training loss', 0)
        total_valid_loss = checkpoint.get('validation loss', 0)
        print("Model checkpoint has been loaded")

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=f"/content/tensorboards/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}")

    # Evaluation-only mode
    if TRAINING == 0:
        test_unweighted_recall, test_weighted_recall, test_accuracy = validation(
            model,
            tqdm(test_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Test", leave=False),
            loss_fn,
            device
        )
        print(f"Epoch {epoch}/{N_EPOCHS}, Test Recall (unweighted): {test_unweighted_recall}, Recall (weighted): {test_weighted_recall}, Accuracy: {test_accuracy}\n")

    else:
        # Training and validation mode
        for epoch in range(epoch, N_EPOCHS+1):
            # Train step - adjust to returned values
            train_unweighted_recall, train_weighted_recall, train_accuracy = train(
                model,
                tqdm(train_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Training", leave=False),
                optimizer,
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Training Recall (unweighted): {train_unweighted_recall}, Recall (weighted): {train_weighted_recall}, Accuracy: {train_accuracy}")

            # Log training metrics
            writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Train", train_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Train", train_weighted_recall, epoch)

            # Validation step - adjust to returned values
            valid_unweighted_recall, valid_weighted_recall, valid_accuracy = validation(
                model,
                tqdm(valid_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Validation", leave=False),
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Validation Recall (unweighted): {valid_unweighted_recall}, Recall (weighted): {valid_weighted_recall}, Accuracy: {valid_accuracy}\n")

            # Log validation metrics
            writer.add_scalar("Accuracy/Validation", valid_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Validation", valid_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Validation", valid_weighted_recall, epoch)

            writer.flush()

            # Save model checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'training loss': train_unweighted_recall,  # Adjust as needed
                'validation loss': valid_unweighted_recall  # Adjust as needed
            }, file_path)

            print(f"Model has been saved after epoch: {epoch}\n")

        # Close TensorBoard writer
        writer.close()


In [26]:
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_train_loss, correct_predictions, total_predictions = 0, 0, 0
    all_actual_labels, all_predictions = [], []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    return unweighted_recall, correct_predictions / total_predictions, total_train_loss / len(train_loader)

In [27]:
# Define the validation function
def validation(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    unweighted_recall = recall_score(all_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_labels, all_predictions, average='weighted')
    accuracy = sum(np.array(all_labels) == np.array(all_predictions)) / len(all_labels)

    return unweighted_recall, weighted_recall, accuracy

In [28]:
# Define the Args class with instance variables
class Args:
    def __init__(self):
        self.N_EPOCHS = 10
        self.LEARNING_RATE = 1e-5
        self.BATCH_SIZE = 8
        self.SESSION = 1
        self.TRAINING = 1
        self.PARADIGM = "FT"
        self.LANGUAGE = "FR"
        self.LINGUALITY = "Monolingual"

In [29]:
def main(args_dict):
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)

    # Access values using args_dict
    session = args_dict["SESSION"]
    batch_size = args_dict["BATCH_SIZE"]
    learning_rate = args_dict["LEARNING_RATE"]

    # Load the CaFE dataset
    label2id, id2label, ds = cafe(session)

    # Initialize feature extractor
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

    # Preprocess dataset
    encoded_audio = ds.map(lambda example: preprocess_function(example, feature_extractor), remove_columns="audio", batched=True)

    # Initialize DataLoaders
    train_loader = DataLoader(encoded_audio["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(encoded_audio["test"], batch_size=batch_size, collate_fn=collate_fn)
    valid_loader = DataLoader(encoded_audio["dev"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Model and optimizer
    model = model_ft(label2id, id2label, len(label2id), device="cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # Run pipeline
    pipeline_ft(model, train_loader, valid_loader, test_loader, args_dict, device="cuda" if torch.cuda.is_available() else "cpu")


In [31]:
!pip install transformers datasets evaluate
!pip install soundfile
import torch
import json
import librosa
import pandas as pd
import numpy as np
import warnings
import random
import os
import argparse
import glob
from sklearn.metrics import recall_score, accuracy_score, precision_recall_fscore_support, confusion_matrix
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset, DatasetDict, Audio, load_dataset
from transformers import (
    Wav2Vec2FeatureExtractor,
    AutoModelForAudioClassification,
    Trainer,
    TrainingArguments,
    Wav2Vec2Processor
)
warnings.filterwarnings("ignore")

In [ ]:
# Create an instance of Args and pass it as a dictionary to main
args = Args()
main(args.__dict__)

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/10, Training:  21%|██        | 11/53 [14:34<54:34, 77.96s/it]